In [589]:
!pip install tmdbv3api

In [537]:
# Importo las librerías necesarias
import pandas as pd
import numpy as np
import time
import ast
import math
import statistics
import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates 
import seaborn as sns
import time
from collections import Counter

from tmdbv3api import TMDb
from tmdbv3api import Movie
import json
import requests

In [538]:
"""
Dado un año, devuelve la film decade
ej: 1906 -> 1900
    1988 -> 1980
"""
def GetDecade(year):
    film_decade = list(np.arange(1900,2040,10))
    interval_years_decade = list(zip(film_decade[:-1],film_decade[1:]))
    for index_decade, interval_decade in list(enumerate(interval_years_decade)):
        if (year in np.arange(interval_decade[0],interval_decade[1])):
            return film_decade[index_decade]
        
"""
Dado un dataframe, chequea valores nan
Si existen nans, devuelve la columna y la cantidad de nans
"""
def Chequearnan(df):
    for col in df.columns:
        nans = df[col].isna().sum()
        if nans > 0:
            print(col, nans)

In [759]:
# Levanto nuestra data y la concateno
filename = 'Data/netflix/Julian/data_julian_complete.csv'
df_ju = pd.read_csv(filename)

filename = 'Data/netflix/Ivan/data_ivan_complete.csv'
df_iv = pd.read_csv(filename)

filename = 'Data/netflix/Dai/data_daiu_complete.csv'
df_dai = pd.read_csv(filename)

In [760]:
df = pd.concat([df_dai, df_iv, df_ju])

In [761]:
df.head()

,Title,Duration,Usuario,Rating Type,Star Value,Thumbs Value,imdbId,movieId,genres_x,tmdbId,primaryTitle,genres_y,startYear,runtimeMinutes,averageRating,numVotes,directors,writers
0,The Princess Diaries 2: Royal Engagement,485.916667,Daiu,star,5.0,2.0,368933.0,8808.0,"['Comedy', 'Romance']",11130.0,The Princess Diaries 2: Royal Engagement,"['Comedy', 'Family', 'Romance']",2004.0,113.0,5.8,80004.0,['Garry Marshall'],['Meg Cabot' 'Gina Wendkos' 'Shonda Rhimes']
1,The Proposal,426.050000,Daiu,star,5.0,2.0,1041829.0,69406.0,"['Comedy', 'Romance']",18240.0,The Proposal,"['Comedy', 'Drama', 'Romance']",2009.0,108.0,6.7,322122.0,['Anne Fletcher'],['Peter Chiarelli']
2,About Time,380.600000,Daiu,star,5.0,2.0,2194499.0,104374.0,"['Drama', 'Fantasy', 'Romance']",122906.0,About Time,"['Comedy', 'Drama', 'Fantasy']",2013.0,123.0,7.8,338543.0,['Richard Curtis'],['Richard Curtis']
3,The Princess Diaries,352.933333,Daiu,star,5.0,2.0,247638.0,4700.0,"['Children', 'Comedy', 'Romance']",9880.0,The Princess Diaries,"['Comedy', 'Family', 'Romance']",2001.0,115.0,6.3,141860.0,['Garry Marshall'],['Meg Cabot' 'Gina Wendkos']
4,The Kissing Booth,315.416667,Daiu,star,3.0,NaN,3799232.0,187613.0,"['Comedy', 'Romance']",454983.0,The Kissing Booth,"['Comedy', 'Romance']",2018.0,105.0,5.9,81898.0,['Vince Marcello'],['Vince Marcello' 'Beth Reekles']


In [762]:
df.info() # No tengo nulos en 'Star Value'

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558 entries, 0 to 38
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Title           558 non-null    object 
 1   Duration        471 non-null    float64
 2   Usuario         558 non-null    object 
 3   Rating Type     313 non-null    object 
 4   Star Value      558 non-null    float64
 5   Thumbs Value    312 non-null    float64
 6   imdbId          558 non-null    float64
 7   movieId         491 non-null    float64
 8   genres_x        491 non-null    object 
 9   tmdbId          491 non-null    float64
 10  primaryTitle    549 non-null    object 
 11  genres_y        556 non-null    object 
 12  startYear       557 non-null    float64
 13  runtimeMinutes  557 non-null    float64
 14  averageRating   549 non-null    float64
 15  numVotes        549 non-null    float64
 16  directors       549 non-null    object 
 17  writers         549 non-null    obje

In [763]:
# Descarto columnas
df = df.loc[:,['Usuario', 'Title', 'primaryTitle', 'imdbId', 'Star Value', 'genres_x', 'genres_y',
               'startYear', 'averageRating', 'numVotes', 'directors']]

df['film_decade'] = df.startYear.apply(lambda x: GetDecade(x))

df.head()

,Usuario,Title,primaryTitle,imdbId,Star Value,genres_x,genres_y,startYear,averageRating,numVotes,directors,film_decade
0,Daiu,The Princess Diaries 2: Royal Engagement,The Princess Diaries 2: Royal Engagement,368933.0,5.0,"['Comedy', 'Romance']","['Comedy', 'Family', 'Romance']",2004.0,5.8,80004.0,['Garry Marshall'],2000.0
1,Daiu,The Proposal,The Proposal,1041829.0,5.0,"['Comedy', 'Romance']","['Comedy', 'Drama', 'Romance']",2009.0,6.7,322122.0,['Anne Fletcher'],2000.0
2,Daiu,About Time,About Time,2194499.0,5.0,"['Drama', 'Fantasy', 'Romance']","['Comedy', 'Drama', 'Fantasy']",2013.0,7.8,338543.0,['Richard Curtis'],2010.0
3,Daiu,The Princess Diaries,The Princess Diaries,247638.0,5.0,"['Children', 'Comedy', 'Romance']","['Comedy', 'Family', 'Romance']",2001.0,6.3,141860.0,['Garry Marshall'],2000.0
4,Daiu,The Kissing Booth,The Kissing Booth,3799232.0,3.0,"['Comedy', 'Romance']","['Comedy', 'Romance']",2018.0,5.9,81898.0,['Vince Marcello'],2010.0


In [764]:
def corregir_genres_y(genres):
    
    if type(genres)==str and ',' in genres:
        genres = genres[2:len(genres)-2].split("', '")
        
    elif type(genres)==str and '[' in genres:
        genres = [genres[2:-2]]
        
    elif type(genres)==str:
        genres = [genres]
    
    return genres

In [765]:
# Convierto los formatos para uso más fácil
df['genres_x'] = df['genres_x'].apply(lambda x: ast.literal_eval(x) if type(x)==str else x)
df['genres_y'] = df['genres_y'].apply(lambda x: corregir_genres_y(x))
df['directors'] = df['directors'].apply(lambda x: x[2:len(x)-2].split("' '") if type(x)==str else x)

In [766]:
df.head(3)

,Usuario,Title,primaryTitle,imdbId,Star Value,genres_x,genres_y,startYear,averageRating,numVotes,directors,film_decade
0,Daiu,The Princess Diaries 2: Royal Engagement,The Princess Diaries 2: Royal Engagement,368933.0,5.0,"[Comedy, Romance]","[Comedy, Family, Romance]",2004.0,5.8,80004.0,[Garry Marshall],2000.0
1,Daiu,The Proposal,The Proposal,1041829.0,5.0,"[Comedy, Romance]","[Comedy, Drama, Romance]",2009.0,6.7,322122.0,[Anne Fletcher],2000.0
2,Daiu,About Time,About Time,2194499.0,5.0,"[Drama, Fantasy, Romance]","[Comedy, Drama, Fantasy]",2013.0,7.8,338543.0,[Richard Curtis],2010.0


In [767]:
# Completo los Nan de primaryTitle con los de Title y elimino la segunda
df['primaryTitle'] = df['primaryTitle'].fillna(df['Title'])
df.drop(columns=['Title'], inplace=True)

In [768]:
# Veamos la que tienen nan en startYear y  film_decade y completemosla a mano
df[df['startYear'].isna()] # No tiene info - la voy a eliminar
df.drop([217],inplace=True)
df = df.reset_index(drop=True)

In [769]:
Chequearnan(df)

genres_x 66
genres_y 1
averageRating 8
numVotes 8
directors 8


In [770]:
# Veamos esas 8 que tienen Nan en 'averageRating', 'numVotes' y 'directors'
df[df['averageRating'].isna()] # Las voy a utilizar para las features que me sirvan

,Usuario,primaryTitle,imdbId,Star Value,genres_x,genres_y,startYear,averageRating,numVotes,directors,film_decade
186,Daiu,The Cutting Edge: Going for the Gold,463953.0,4.0,"[Comedy, Drama, Romance]","[Drama, Action, Comedy, Romance]",2006.0,NaN,NaN,NaN,2000.0
194,Daiu,Remember Sunday,2770480.0,3.0,"[Drama, Romance]","[Drama, Romance]",2013.0,NaN,NaN,NaN,2010.0
261,Daiu,Blue Lagoon: The Awakening,2287663.0,1.0,[Romance],"[Drama, Romance, Adventure]",2012.0,NaN,NaN,NaN,2010.0
284,Daiu,Date with Love,5615976.0,2.0,"[Comedy, Romance]","[Comedy, Romance]",2016.0,NaN,NaN,NaN,2010.0
306,Daiu,Autumn Dreams,5066870.0,2.0,[Romance],"[Comedy, Romance]",2015.0,NaN,NaN,NaN,2010.0
321,Daiu,Cruel Intentions 3,391891.0,1.0,NaN,[Drama],2004.0,NaN,NaN,NaN,2000.0
329,Daiu,Life Is Beautiful,1433815.0,4.0,NaN,"[Drama, Romance]",2009.0,NaN,NaN,NaN,2000.0
393,Daiu,Radio Rebel,2008633.0,1.0,[Drama],"[Drama, Comedy, Family]",2012.0,NaN,NaN,NaN,2010.0


In [771]:
# Veamos el que tiene Nan en 'generes_y'
df[df['genres_y'].isna()]
df.drop([163],inplace=True)
df = df.reset_index(drop=True)

In [772]:
Chequearnan(df)

genres_x 65
averageRating 8
numVotes 8
directors 8


In [773]:
Counter(df.explode('genres_x')['genres_x'])['(no genres listed)']

5

In [774]:
# Para 'genres_x' tengo 5 + 65 = 70 sin género
# Para las peliculas que no tengan informacion en genero dejo listas vacias [] en vez de ['non genres listed'] o 'np.nan'
df['genres_x'] = df['genres_x'].apply(lambda x : list() if x == ['(no genres listed)'] else x)
df['genres_x'] = df['genres_x'].fillna(0)
df['genres_x'] = df['genres_x'].apply(lambda x : list() if x==0 else x)

In [775]:
# Verifico
Counter(df.explode('genres_x')['genres_x'])['(no genres listed)']

0

In [776]:
Chequearnan(df)

averageRating 8
numVotes 8
directors 8


In [777]:
df[['genres_x']].explode('genres_x')['genres_x'].unique() 
# acá me figuran nan porque al hacer el explode de las lists vacías queda nan

array(['Comedy', 'Romance', 'Drama', 'Fantasy', 'Children', 'Horror',
       'Mystery', 'Thriller', 'Animation', 'IMAX', 'Action', 'Adventure',
       'Crime', 'Musical', 'Sci-Fi', 'War', nan, 'Western', 'Film-Noir',
       'Documentary'], dtype=object)

In [778]:
df[['genres_y']].explode('genres_y')['genres_y'].unique()

array(['Comedy', 'Family', 'Romance', 'Drama', 'Fantasy', 'Mystery',
       'Thriller', 'Action', 'Adventure', 'Crime', 'Animation', 'Horror',
       'Music', 'History', 'Sci-Fi', 'War', 'Biography', 'Film-Noir',
       'Documentary', 'Musical', 'Sport', 'Western'], dtype=object)

In [779]:
df.explode('genres_y')[df.explode('genres_y')['genres_y']=='Music'].head()
# Que tenga música no quiere decir que sea un musical - podría querer eliminar este género - pero como no lo hicimos en el otro en principio no

,Usuario,primaryTitle,imdbId,Star Value,genres_x,genres_y,startYear,averageRating,numVotes,directors,film_decade
15,Daiu,Center Stage,210616.0,5.0,"[Drama, Musical]",Music,2000.0,6.6,21551.0,[Nicholas Hytner],2000.0
27,Daiu,Music and Lyrics,758766.0,5.0,"[Comedy, Romance]",Music,2007.0,6.5,102642.0,[Marc Lawrence],2000.0
92,Daiu,Magic Mike XXL,2268016.0,4.0,"[Comedy, Drama]",Music,2015.0,5.6,55893.0,[Gregory Jacobs],2010.0
157,Daiu,If I Stay,1355630.0,1.0,[Drama],Music,2014.0,6.7,122117.0,[R.J. Cutler],2010.0
178,Daiu,Dirty Dancing,92890.0,5.0,"[Drama, Musical, Romance]",Music,1987.0,7.0,225980.0,[Emile Ardolino],1980.0


In [780]:
# Creo una columna con el merge de ambas listas de géneros
df['genres_fusion'] = (df['genres_x'] + df['genres_y'] ).apply(lambda x: list(set(x))) # Convierto a set para quedarme con los valores únicos

In [781]:
# Me creo un df solo con las películas y su información
df_movies = df[['primaryTitle', 'imdbId', 'genres_x', 'genres_y', 'startYear', 'averageRating', 'numVotes', 'directors', 'film_decade', 'genres_fusion']]

In [782]:
df_movies.shape[0]

556

In [783]:
df_movies['primaryTitle'].nunique()

530

In [784]:
df_movies = df_movies.drop_duplicates(subset=['primaryTitle'])#.shape[0]

In [785]:
df_movies.shape[0] # ahora tengo una única línea por película

530

In [786]:
# Veamos cuáles son estos géneros
generos = df_movies[['genres_fusion']].explode('genres_fusion')['genres_fusion'].nunique()
print(f'Cantidad de géneros: {generos}')
print('Géneros:')
df_movies[['genres_fusion']].explode('genres_fusion')['genres_fusion'].value_counts()/530*100

Cantidad de géneros: 24
Géneros:


Drama          61.132075
Comedy         46.226415
Romance        37.735849
Thriller       28.867925
Action         18.490566
Crime          18.490566
Adventure      16.415094
Mystery        12.641509
Fantasy        10.000000
Sci-Fi          9.622642
Horror          9.433962
Family          5.849057
Children        5.660377
Biography       4.905660
Animation       3.773585
IMAX            3.773585
Music           3.584906
Musical         3.396226
History         2.264151
Documentary     2.264151
War             1.509434
Sport           0.943396
Film-Noir       0.566038
Western         0.566038
Name: genres_fusion, dtype: float64

In [787]:
# Discrimino los géneros IMAX, News, Adult y Reality-TV (y Sport en este caso)
df['genres_fusion'] = df['genres_fusion'].apply(lambda x : list(set(x).difference(set(['IMAX','News','Adult','Reality-TV','Sport']))))

# Para el df_movies elimino además las mismas que Juli
df_movies['genres_fusion'] = df_movies['genres_fusion'].apply(lambda x : list(set(x).difference(set(['IMAX','Adult','Film-Noir', 'Biography', 'History', 'Family', 'Music', 'Sport', 'News']))))

In [788]:
# Elimino las columnas de género que ya no voy a utilizar
df.drop(columns=['genres_x','genres_y'],inplace=True)

# También para df_movies
df_movies.drop(columns=['genres_x','genres_y'],inplace=True)

####
df.head()

,Usuario,primaryTitle,imdbId,Star Value,startYear,averageRating,numVotes,directors,film_decade,genres_fusion
0,Daiu,The Princess Diaries 2: Royal Engagement,368933.0,5.0,2004.0,5.8,80004.0,[Garry Marshall],2000.0,"[Romance, Family, Comedy]"
1,Daiu,The Proposal,1041829.0,5.0,2009.0,6.7,322122.0,[Anne Fletcher],2000.0,"[Romance, Comedy, Drama]"
2,Daiu,About Time,2194499.0,5.0,2013.0,7.8,338543.0,[Richard Curtis],2010.0,"[Romance, Fantasy, Comedy, Drama]"
3,Daiu,The Princess Diaries,247638.0,5.0,2001.0,6.3,141860.0,[Garry Marshall],2000.0,"[Children, Romance, Family, Comedy]"
4,Daiu,The Kissing Booth,3799232.0,3.0,2018.0,5.9,81898.0,[Vince Marcello],2010.0,"[Romance, Comedy]"


### Nuestros ratings

In [757]:
df['Usuario'] = df['Usuario'].apply(lambda x: 'Daiana' if x == 'Daiu' else x)

In [789]:
df_ratings = df[['Usuario', 'Star Value', 'imdbId']]

In [792]:
df_ratings['Usuario'] = df_ratings['Usuario'].apply(lambda x: 'Daiana' if x == 'Daiu' else x)

<ipython-input-792-9731e1af7531>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ratings['Usuario'] = df_ratings['Usuario'].apply(lambda x: 'Daiana' if x == 'Daiu' else x)


In [794]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Usuario     556 non-null    object 
 1   Star Value  556 non-null    float64
 2   imdbId      556 non-null    float64
dtypes: float64(2), object(1)
memory usage: 13.2+ KB


In [795]:
#df_ratings.to_csv("Data/nuestra/df_ratings_nuestro.csv", index=False)

### Nuestras películas
Me traigo el código de Juli para agregar actores y keywords.

In [699]:
# Cargamos actores
crew_file = 'Data/imdb/credits.csv.zip'
#crew_path = os.path.join(files_dir, crew_file)

df_actors = pd.read_csv(crew_file, compression='zip')

# Renombramos y tiramos la crew
df_actors.drop(['crew'], axis=1, inplace=True)

# Cargamos keywords
keywords_file = 'Data/imdb/keywords.csv.zip'
#keywords_path = os.path.join(files_dir, keywords_file)

df_key = pd.read_csv(keywords_file, compression='zip')

# Cargamos los ids de imdb
id_file = 'Data/imdb/movies_metadata.csv.zip'
#id_path = os.path.join(files_dir, id_file)

df_id = pd.read_csv(id_file, compression='zip')
df_id = df_id[['imdb_id', 'id']]

# Renombramos y pasamos a int
df_id['id'] = df_id['id'].apply(lambda x: int(x) if '-' not in x else x)
df_id.rename({'imdb_id' : 'imdbId'}, axis=1, inplace=True)

C:\Users\DAIANA_NUBI\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [700]:
df_actors = df_actors.merge(df_id, on='id')
df_actors = df_actors.merge(df_key, on='id')

In [701]:
df_actors.dropna(inplace=True)
df_actors['imdbId'] = df_actors['imdbId'].apply(lambda x: x[2:]).astype(int)

In [702]:
df_actors

,cast,id,imdbId,keywords
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862,114709,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...",8844,113497,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...",15602,113228,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...",31357,114885,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...",11862,113041,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...,...,...
46623,"[{'cast_id': 0, 'character': '', 'credit_id': ...",439050,6209470,"[{'id': 10703, 'name': 'tragic love'}]"
46624,"[{'cast_id': 1002, 'character': 'Sister Angela...",111109,2028550,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46625,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...",67758,303758,[]
46626,"[{'cast_id': 2, 'character': '', 'credit_id': ...",227506,8536,[]


In [703]:
# Pasamos todos los strings a objetos de python
df_actors['cast'] = df_actors['cast'].apply(ast.literal_eval)
df_actors['keywords'] = df_actors['keywords'].apply(ast.literal_eval)

In [704]:
# Me voy a quedar con los tres actores principales
df_actors['cast'] = df_actors['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df_actors['cast'] = df_actors['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [705]:
# Limpiamos la columna keywords
df_actors['keywords'] = df_actors['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [706]:
df_actors.drop(['id'], axis=1, inplace=True)
df_actors

,cast,imdbId,keywords
0,"[Tom Hanks, Tim Allen, Don Rickles]",114709,"[jealousy, toy, boy, friendship, friends, riva..."
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",113497,"[board game, disappearance, based on children'..."
2,"[Walter Matthau, Jack Lemmon, Ann-Margret]",113228,"[fishing, best friend, duringcreditsstinger, o..."
3,"[Whitney Houston, Angela Bassett, Loretta Devine]",114885,"[based on novel, interracial relationship, sin..."
4,"[Steve Martin, Diane Keaton, Martin Short]",113041,"[baby, midlife crisis, confidence, aging, daug..."
...,...,...,...
46623,"[Leila Hatami, Kourosh Tahami, Elham Korda]",6209470,[tragic love]
46624,"[Angel Aquino, Perry Dizon, Hazel Orencio]",2028550,"[artist, play, pinoy]"
46625,"[Erika Eleniak, Adam Baldwin, Julie du Page]",303758,[]
46626,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",8536,[]


In [707]:
# Para esto necesito guardarme el imdbId de nuestras películas
df_movies.head()

,primaryTitle,imdbId,startYear,averageRating,numVotes,directors,film_decade,genres_fusion
0,The Princess Diaries 2: Royal Engagement,368933.0,2004.0,5.8,80004.0,[Garry Marshall],2000.0,"[Romance, Comedy]"
1,The Proposal,1041829.0,2009.0,6.7,322122.0,[Anne Fletcher],2000.0,"[Romance, Comedy, Drama]"
2,About Time,2194499.0,2013.0,7.8,338543.0,[Richard Curtis],2010.0,"[Romance, Fantasy, Comedy, Drama]"
3,The Princess Diaries,247638.0,2001.0,6.3,141860.0,[Garry Marshall],2000.0,"[Children, Romance, Comedy]"
4,The Kissing Booth,3799232.0,2018.0,5.9,81898.0,[Vince Marcello],2010.0,"[Romance, Comedy]"


In [578]:
df_movies[df_movies['imdbId'].isna()] # todas tienen imdbId

,primaryTitle,imdbId,startYear,averageRating,numVotes,directors,film_decade,genres_fusion


In [580]:
df_movies.shape[0]

530

In [581]:
len(set(df_actors['imdbId']).intersection(set(df_movies['imdbId'])))

402

In [708]:
# Quiero ver cuáles son las que no encontró - 128
ids_no_estan = list((set(df_movies['imdbId']))-set(df_actors['imdbId']))

# Me voy a guardar este pedazo para completarlo
df_completar = df_movies[df_movies['imdbId'].isin(ids_no_estan)] # Algunas capaz son muy nuevas y por eso no están

In [624]:
tmdb = TMDb()
tmdb.api_key = '1b43ba0b165446878468749e2e628f1b'
tmdb_movie = Movie()

In [657]:
'''
imdb_id = 993840.0

imdb_id = str(imdb_id)[:-2]
imdb_id = 'tt' + imdb_id

for i in range(4):
    
    imdb_id_mod = imdb_id[:2] + i * '0' + imdb_id[2:]
    
    print(i, imdb_id_mod)
'''

0 tt993840
1 tt0993840
2 tt00993840
3 tt000993840


In [675]:
def complete_info_cast(imdb_id):
    
    imdb_id = str(imdb_id)[:-2]
    imdb_id = 'tt' + imdb_id
    
    for i in range(3):
    
        imdb_id_mod = imdb_id[:2] + i * '0' + imdb_id[2:]
    
        # Busco el id de tmdb utilizando el id de imdb
        movie_info = requests.get('https://api.themoviedb.org/3/find/{}?api_key={}&language=en-US&external_source=imdb_id'.format(imdb_id_mod,tmdb.api_key))
        movie_info = movie_info.json()
        
        # Si el id de imdb era correcto 
        if movie_info['movie_results']!=list():
            tmdb_id = movie_info['movie_results'][0]['id']
        
            movie_credits = requests.get('https://api.themoviedb.org/3/movie/{}/credits?api_key={}&language=en-US'.format(tmdb_id,tmdb.api_key))
            movie_credits = movie_credits.json()
        
            cast = movie_credits['cast']
            break
        
        else:
            cast = []
    
    return cast        

In [685]:
def complete_info_keywords(imdb_id):
    
    imdb_id = str(imdb_id)[:-2]
    imdb_id = 'tt' + imdb_id
    
    for i in range(3):
    
        imdb_id_mod = imdb_id[:2] + i * '0' + imdb_id[2:]
    
        # Busco el id de tmdb utilizando el id de imdb
        movie_info = requests.get('https://api.themoviedb.org/3/find/{}?api_key={}&language=en-US&external_source=imdb_id'.format(imdb_id_mod,tmdb.api_key))
        movie_info = movie_info.json()
    
        # Si el id de imdb era correcto 
        if movie_info['movie_results']!=list():
            
            tmdb_id = movie_info['movie_results'][0]['id']
        
            movie_keywords = requests.get('https://api.themoviedb.org/3/movie/{}/keywords?api_key={}'.format(tmdb_id,tmdb.api_key))
            movie_keywords = movie_keywords.json()
        
            keywords = movie_keywords['keywords']
            break
        
        else:
            keywords = []
    
    return keywords

In [724]:
df_completar['keywords'] = df_completar['imdbId'].apply(lambda x: complete_info_keywords(x))

<ipython-input-724-556051ea2587>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completar['keywords'] = df_completar['imdbId'].apply(lambda x: complete_info_keywords(x))


In [725]:
df_completar['cast'] = df_completar['imdbId'].apply(lambda x: complete_info_cast(x))

<ipython-input-725-10d9f147ec38>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completar['cast'] = df_completar['imdbId'].apply(lambda x: complete_info_cast(x))


In [726]:
df_completar.head()

,primaryTitle,imdbId,startYear,averageRating,numVotes,directors,film_decade,genres_fusion,keywords,cast
4,The Kissing Booth,3799232.0,2018.0,5.9,81898.0,[Vince Marcello],2010.0,"[Romance, Comedy]","[{'id': 818, 'name': 'based on novel or book'}...","[{'adult': False, 'gender': 1, 'id': 125025, '..."
42,Army of the Dead,993840.0,2021.0,5.7,167530.0,[Zack Snyder],2020.0,"[Action, Horror, Crime]","[{'id': 585, 'name': 'casino'}, {'id': 1953, '...","[{'adult': False, 'gender': 2, 'id': 543530, '..."
45,Don't Look Up,11286314.0,2021.0,7.2,504347.0,[Adam McKay],2020.0,"[Comedy, Sci-Fi, Drama]","[{'id': 310, 'name': 'artificial intelligence'...","[{'adult': False, 'gender': 1, 'id': 72129, 'k..."
46,Marriage Story,7653254.0,2019.0,7.9,295589.0,[Noah Baumbach],2010.0,"[Romance, Comedy, Drama]","[{'id': 242, 'name': 'new york city'}, {'id': ...","[{'adult': False, 'gender': 2, 'id': 1023139, ..."
47,The Devil All the Time,7395114.0,2020.0,7.1,128515.0,[Antonio Campos],2020.0,"[Thriller, Crime, Drama]","[{'id': 236, 'name': 'suicide'}, {'id': 798, '...","[{'adult': False, 'gender': 2, 'id': 1136406, ..."


In [727]:
# Hago las mismas transformaciones que Juli

# Me voy a quedar con los tres actores principales
df_completar['cast'] = df_completar['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
df_completar['cast'] = df_completar['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

# Limpiamos la columna keywords
df_completar['keywords'] = df_completar['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

<ipython-input-727-f96bf211b423>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completar['cast'] = df_completar['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
<ipython-input-727-f96bf211b423>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completar['cast'] = df_completar['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
<ipython-input-727-f96bf211b423>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [728]:
df_completar.head()

,primaryTitle,imdbId,startYear,averageRating,numVotes,directors,film_decade,genres_fusion,keywords,cast
4,The Kissing Booth,3799232.0,2018.0,5.9,81898.0,[Vince Marcello],2010.0,"[Romance, Comedy]","[based on novel or book, crush, los angeles, c...","[Joey King, Joel Courtney, Jacob Elordi]"
42,Army of the Dead,993840.0,2021.0,5.7,167530.0,[Zack Snyder],2020.0,"[Action, Horror, Crime]","[casino, vault, mercenary, tiger, safe, zombie...","[Dave Bautista, Ella Purnell, Omari Hardwick]"
45,Don't Look Up,11286314.0,2021.0,7.2,504347.0,[Adam McKay],2020.0,"[Comedy, Sci-Fi, Drama]","[artificial intelligence, the white house, ast...","[Jennifer Lawrence, Leonardo DiCaprio, Meryl S..."
46,Marriage Story,7653254.0,2019.0,7.9,295589.0,[Noah Baumbach],2010.0,"[Romance, Comedy, Drama]","[new york city, parent child relationship, hus...","[Adam Driver, Scarlett Johansson, Laura Dern]"
47,The Devil All the Time,7395114.0,2020.0,7.1,128515.0,[Antonio Campos],2020.0,"[Thriller, Crime, Drama]","[suicide, sheriff, based on novel or book, sma...","[Tom Holland, Bill Skarsgård, Riley Keough]"


In [717]:
df_actors.shape[0] # 46611
#df_actors['imdbId'].nunique() # 45415

46611

In [723]:
df_actors[df_actors['imdbId'].isna()] # Todos tienen imdbId

,cast,imdbId,keywords


In [721]:
df_actors[df_actors['imdbId'].duplicated()] # Acá hay duplicados

,cast,imdbId,keywords
677,"[Corinna Harfouch, Meret Becker, August Zirner]",111613,"[anti-communism, political, woman director]"
678,"[Corinna Harfouch, Meret Becker, August Zirner]",111613,"[anti-communism, political, woman director]"
679,"[Corinna Harfouch, Meret Becker, August Zirner]",111613,"[anti-communism, political, woman director]"
680,"[Corinna Harfouch, Meret Becker, August Zirner]",111613,"[anti-communism, political, woman director]"
681,"[Corinna Harfouch, Meret Becker, August Zirner]",111613,"[anti-communism, political, woman director]"
...,...,...,...
37505,"[Chen Kun, Huang Bo, Shu Qi]",4276752,"[love triangle, past, zombie, tomb, grave robb..."
37507,[Trevor Noah],2073693,[]
37509,[Trevor Noah],1552639,[]
37511,"[Amy Jo Johnson, Brett Cullen, Wendie Malick]",119881,[sport]


In [731]:
# Hago el merge solo con los que encontró 
#df_merge = df_movies[~df_movies['imdbId'].isin(ids_no_estan)].merge(df_actors.drop_duplicates(subset=['imdbId']), on='imdbId', how='left')

# Y finalmente concateno las dos partes
df_movies_complete = pd.concat([df_merge, df_completar])

df_movies_complete.shape[0]

530

In [732]:
df_movies_complete

,primaryTitle,imdbId,startYear,averageRating,numVotes,directors,film_decade,genres_fusion,cast,keywords
0,The Princess Diaries 2: Royal Engagement,368933.0,2004.0,5.8,80004.0,[Garry Marshall],2000.0,"[Romance, Comedy]","[Anne Hathaway, Julie Andrews, Héctor Elizondo]","[coronation, duty, marriage, falling in love]"
1,The Proposal,1041829.0,2009.0,6.7,322122.0,[Anne Fletcher],2000.0,"[Romance, Comedy, Drama]","[Sandra Bullock, Ryan Reynolds, Mary Steenburgen]","[fictitious marriage, deportation, immigration..."
2,About Time,2194499.0,2013.0,7.8,338543.0,[Richard Curtis],2010.0,"[Romance, Fantasy, Comedy, Drama]","[Domhnall Gleeson, Rachel McAdams, Bill Nighy]","[london england, father son relationship, time..."
3,The Princess Diaries,247638.0,2001.0,6.3,141860.0,[Garry Marshall],2000.0,"[Children, Romance, Comedy]","[Anne Hathaway, Julie Andrews, Heather Matarazzo]","[heir to the throne, grandmother granddaughter..."
4,The Sixth Sense,167404.0,1999.0,8.2,970632.0,[M. Night Shyamalan],1990.0,"[Horror, Mystery, Thriller, Drama]","[Bruce Willis, Toni Collette, Olivia Williams]","[child abuse, sense of guilt, loss of child, c..."
...,...,...,...,...,...,...,...,...,...,...
534,Annihilation,2798920.0,2018.0,6.8,315042.0,[Alex Garland],2010.0,"[Drama, Thriller, Horror, Adventure, Mystery, ...","[Natalie Portman, Jennifer Jason Leigh, Gina R...","[coma, based on novel or book, depression, flo..."
535,Gold,6020800.0,2022.0,5.4,6481.0,[Anthony Hayes],2020.0,"[Action, Thriller]","[Zac Efron, Anthony Hayes, Susie Porter]",[]
542,SHOT! The Psycho-Spiritual Mantra of Rock,3044562.0,2016.0,6.8,752.0,[Barney Clay],2010.0,[Documentary],"[Karen O, Tunde Adebimpe, Father John Misty]","[new york city, drug abuse, 1970s, punk rock, ..."
545,Extinction,3201640.0,2018.0,5.8,52434.0,[Ben Young],2010.0,"[Action, Drama, Sci-Fi]","[Michael Peña, Lizzy Caplan, Israel Broussard]","[space, alien, mechanic, alien invasion, space..."


In [734]:
df_movies_complete.isna().any()

primaryTitle     False
imdbId           False
startYear        False
averageRating     True
numVotes          True
directors         True
film_decade      False
genres_fusion    False
cast             False
keywords         False
dtype: bool

In [735]:
df_movies_complete[df_movies_complete['directors'].isna()] # Consultar con Juli si no pasa nada que tenga Nan en directors

,primaryTitle,imdbId,startYear,averageRating,numVotes,directors,film_decade,genres_fusion,cast,keywords
138,The Cutting Edge: Going for the Gold,463953.0,2006.0,NaN,NaN,NaN,2000.0,"[Action, Romance, Comedy, Drama]","[Christy Carlson Romano, Ross Thomas, Scott Th...","[olympic games, sport, figure skating]"
142,Remember Sunday,2770480.0,2013.0,NaN,NaN,NaN,2010.0,"[Romance, Drama]","[Alexis Bledel, Zachary Levi, Merritt Wever]",[memory loss]
184,Blue Lagoon: The Awakening,2287663.0,2012.0,NaN,NaN,NaN,2010.0,"[Romance, Adventure, Drama]","[Indiana Evans, Brenton Thwaites, Denise Richa...","[stranded, marooned, teenager, high school stu..."
200,Date with Love,5615976.0,2016.0,NaN,NaN,NaN,2010.0,"[Romance, Comedy]","[Bailee Madison, Shenae Grimes, Andrew W. Walker]","[prom, teen comedy]"
218,Autumn Dreams,5066870.0,2015.0,NaN,NaN,NaN,2010.0,"[Romance, Comedy]","[Jill Wagner, Colin Egglesfield, Michael Karl ...",[]
225,Cruel Intentions 3,391891.0,2004.0,NaN,NaN,NaN,2000.0,[Drama],"[Kerr Smith, Kristina Anapau, Nathan Wethering...","[female nudity, sex, manipulation, sequel, tee..."
286,Radio Rebel,2008633.0,2012.0,NaN,NaN,NaN,2010.0,"[Comedy, Drama]","[Debby Ryan, Sarena Parmar, Adam DiMarco]","[shyness, jealousy, secret identity, radio, st..."
328,Life Is Beautiful,1433815.0,2009.0,NaN,NaN,NaN,2000.0,"[Romance, Drama]","[Robert de Hoog, Katja Herbers, Sylvia Hoeks]",[]


In [745]:
# Lo voy a convertir a lista vacía como en las demás columnas
df_movies_complete['directors'] = df_movies_complete['directors'].apply(lambda x: x if type(x)==list else list())

In [747]:
df_movies_complete[df_movies_complete['primaryTitle']=='The Cutting Edge: Going for the Gold']

,primaryTitle,imdbId,startYear,averageRating,numVotes,directors,film_decade,genres_fusion,cast,keywords
138,The Cutting Edge: Going for the Gold,463953.0,2006.0,NaN,NaN,[],2000.0,"[Action, Romance, Comedy, Drama]","[Christy Carlson Romano, Ross Thomas, Scott Th...","[olympic games, sport, figure skating]"


In [746]:
df_movies_complete[df_movies_complete['directors'].isna()]

,primaryTitle,imdbId,startYear,averageRating,numVotes,directors,film_decade,genres_fusion,cast,keywords


In [752]:
# Me guardo la info completa de nuestras películas
#df_movies_complete.to_csv("Data/nuestra/df_movies_nuestro.csv", index=False)

### Features de usuario

In [457]:
# Agrego columnas de rating: promedio, cantidad de peliculas con rating y la suma.
df_users = df.groupby('Usuario').agg(rating_mean = ('Star Value','mean'),
                                      rating_std = ('Star Value','std'),
                                      rating_median = ('Star Value','median'),
                                      movies_count = ('Star Value','count')).reset_index()

In [458]:
df_users

,Usuario,rating_mean,rating_std,rating_median,movies_count
0,Daiu,2.834081,1.215741,3.0,446
1,Ivan,2.873239,1.443503,3.0,71
2,Julian,3.115385,1.166763,3.0,39


In [459]:
# Agrego año favorito
df_año_fav = df.groupby('Usuario').startYear.apply(lambda x : x.mode()).reset_index()

In [461]:
df_año_fav = df_año_fav[df_año_fav['level_1']==0].drop(columns=['level_1'])

In [463]:
df_users = df_users.merge(df_año_fav, on='Usuario').rename(columns={'startYear':'year_fav'})

In [464]:
# Agrego decada favorita
df_dec_fav = df.groupby('Usuario').film_decade.apply(lambda x : x.mode()).reset_index().rename(columns={'film_decade': 'decade_fav'})

In [466]:
df_dec_fav = df_dec_fav[df_dec_fav['level_1']==0].drop(columns=['level_1'])

In [468]:
df_users = df_users.merge(df_dec_fav, on='Usuario').rename(columns={'startYear':'year_fav'})

In [469]:
# Agrego cantidad de peliculas puntuadas de la decada favorita
df_dec_fav_count = df.groupby('Usuario').film_decade.apply(lambda x : x.value_counts().head(1).item()).reset_index().rename(columns={'film_decade': 'decade_fav_count'})

In [472]:
df_users = df_users.merge(df_dec_fav_count, on='Usuario')

In [474]:
# Agrego proporcion de peliculas puntuadas de decada favorita
df_users['decade_fav_per'] = df_users['decade_fav_count'] / df_users['movies_count']

In [475]:
df_users

,Usuario,rating_mean,rating_std,rating_median,movies_count,year_fav,decade_fav,decade_fav_count,decade_fav_per
0,Daiu,2.834081,1.215741,3.0,446,2013.0,2010.0,252,0.565022
1,Ivan,2.873239,1.443503,3.0,71,2018.0,2010.0,40,0.563380
2,Julian,3.115385,1.166763,3.0,39,2014.0,2010.0,24,0.615385


In [476]:
# Agrego director favorito
df_dir_fav = df[['Usuario', 'directors']].explode('directors').dropna(subset=['directors']).groupby('Usuario')['directors'].apply(lambda x : statistics.mode(x)).reset_index()

In [477]:
df_users = df_users.merge(df_dir_fav, on='Usuario').rename(columns={'directors':'director_fav'})

In [479]:
# Agrego cantidad de peliculas puntuadas del director favorito
df_count_fav_directors = df[['Usuario', 'directors']].explode('directors').groupby('Usuario').apply(lambda x : x.value_counts().head(1).item()).reset_index()

In [480]:
df_count_fav_directors.rename(columns={0: 'director_fav_count'},inplace=True)

In [482]:
df_users = df_users.merge(df_count_fav_directors, on='Usuario')

In [483]:
# Agrego proporcion de peliculas puntuadas de director favorito
df_users['director_fav_per'] = df_users['director_fav_count'] / df_users['movies_count']

In [484]:
df_users

,Usuario,rating_mean,rating_std,rating_median,movies_count,year_fav,decade_fav,decade_fav_count,decade_fav_per,director_fav,director_fav_count,director_fav_per
0,Daiu,2.834081,1.215741,3.0,446,2013.0,2010.0,252,0.565022,Quentin Tarantino,5,0.011211
1,Ivan,2.873239,1.443503,3.0,71,2018.0,2010.0,40,0.563380,Quentin Tarantino,3,0.042254
2,Julian,3.115385,1.166763,3.0,39,2014.0,2010.0,24,0.615385,Richard Linklater,2,0.051282


In [489]:
df_users = df_users.merge(df.groupby('Usuario').agg(year_median = ('startYear','median'),
                                                     year_std = ('startYear','std'),
                                                     film_decade_std = ('film_decade','std'),
                                                     film_decade_median = ('film_decade','median')).reset_index(), on='Usuario')

In [490]:
df_users

,Usuario,rating_mean,rating_std,rating_median,movies_count,year_fav,decade_fav,decade_fav_count,decade_fav_per,director_fav,director_fav_count,director_fav_per,year_median,year_std,film_decade_std,film_decade_median
0,Daiu,2.834081,1.215741,3.0,446,2013.0,2010.0,252,0.565022,Quentin Tarantino,5,0.011211,2012.0,9.080510,9.537102,2010.0
1,Ivan,2.873239,1.443503,3.0,71,2018.0,2010.0,40,0.563380,Quentin Tarantino,3,0.042254,2012.0,12.977538,12.487418,2010.0
2,Julian,3.115385,1.166763,3.0,39,2014.0,2010.0,24,0.615385,Richard Linklater,2,0.051282,2014.0,13.725672,14.170735,2010.0


In [751]:
#df_users.to_csv("Data/nuestra/df_info_users_nuestro.csv", index=False)